# Node

> TODO fill in description

In [ ]:
#| default_exp node

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from collections import deque
from types import MappingProxyType
from typing import Type, Optional, Union, Any, Tuple, Dict, Coroutine, List, Hashable, Deque
import traceback

import fbdev
from fbdev.utils import StateHandler, StateView, StateCollection, EventHandler, EventCollection, AttrContainer, await_multiple_events, await_any_event, TaskManager, is_in_event_loop, AddressableMixin
from fbdev.packet import Packet, PacketRegistry, PacketHandler
from fbdev.port import PortType, PortSpec, ConfigPortSpec, PortTypeSpec, PortSpecCollection, BasePort, InputPort, ConfigPort, OutputPort, PortCollection
from fbdev.component import BaseComponent, ComponentFactory, PortSpec, BasePort, InputPort, OutputPort, PortCollection, PortType
from fbdev.graph import EdgeSpec, NodeSpec, Graph, ReadonlyGraph
from fbdev.exceptions import ComponentError, NodeError, LostPacketError

In [ ]:
#|hide
from fbdev.component import func_component

In [ ]:
#|hide
show_doc(fbdev.node.Edge)

---

### Edge

>      Edge (edge_spec:fbdev.graph.EdgeSpec,
>            parent_graph_process:fbdev.node.GraphComponentFactory)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class Edge(AddressableMixin):
    _address_separator = '|'
    
    def __init__(self,
                 edge_spec:EdgeSpec,
                 parent_graph_process:GraphComponentFactory):
        self._graph_elem_type = 'EDGE'
        
        self._parent_graph_process:BaseComponent = parent_graph_process
        self._edge_spec:EdgeSpec = edge_spec
                
        self._states:StateCollection = StateCollection()
        self._states._add_state(StateHandler("enabled", True), readonly=False)
        self._states._add_state(StateHandler("empty", True), readonly=True)
        self._states._add_state(StateHandler("full", False), readonly=True)
        
        self._events:EventCollection = EventCollection()
        self._events._add_event(EventHandler("has_changed"))
        
        self._maxsize:int = edge_spec.maxsize
        self._packets:Deque[Packet] = deque(maxlen=edge_spec.maxsize)

    @property
    def parent_graph_process(self): return self._parent_graph_process
    @property
    def edge_spec(self): return self._edge_spec    
    @property
    def states(self): return self._states
    @property
    def events(self): return self._events
    
    @property
    def id(self): return self._edge_spec.id
    @property
    def rich_id(self): return f"{self.id}"
    @property
    def parent(self): return self.parent_graph_process._parent_node
 
    @property
    def tail_connection(self):
        if self.id in self.parent_graph_process.graph.edge_to_tail_connections:
            target_id = self.parent_graph_process.graph.edge_to_tail_connections[self.id]
            if target_id != Graph.GRAPH_ID:
                return self.parent_graph_process.nodes[target_id]
            else:
                return self.parent_graph_process
        else:
            return None
    @property
    def head_connection(self):
        if self.id in self.parent_graph_process.graph.edge_to_head_connections:
            target_id = self.parent_graph_process.graph.edge_to_head_connections[self.id]
            if target_id != Graph.GRAPH_ID:
                return self.parent_graph_process.nodes[target_id]
            else:
                return self.parent_graph_process
        else:
            return None

    @property
    def maxsize(self): return self.edge_spec.maxsize
    @property
    def finite_maxsize(self): return self.edge_spec.finite_maxsize
        
    def num_packets(self): return len(self._packets)
        
    def _update_status_events(self):        
        if len(self._packets) > 0: self.states._empty.set(False)
        else: self.states._empty.set(True)
        if len(self._packets) == self.maxsize and self.maxsize is not None: self.states._full.set(True)
        else: self.states._full.set(False)
    
    def _load(self, packet:Packet):
        if self.states.full.get(): raise RuntimeError("Edge is full.")
        self._packets.append(packet)
        self._update_status_events()
        self.events.has_changed._trigger()
        
    def _unload(self):
        if self.states.empty.get(): raise RuntimeError("Edge is empty.")
        packet = self._packets.popleft()
        self._update_status_events()
        self.events.has_changed._trigger()
        return packet
    
    def _load_left(self, packet: Packet):
        if self.states.full.get(): raise RuntimeError("Edge is full.")
        self._packets.appendleft(packet)
        self._update_status_events()
        self.events.has_changed._trigger()
    
    def _unload_right(self):
        if self.states.empty.get(): raise RuntimeError("Edge is empty.")
        packet = self._packets.pop()
        self._update_status_events()
        self.events.has_changed._trigger()
        return packet
        
    def _unload_all(self):
        if self.states.empty.get(): raise RuntimeError("Edge is empty.")
        packets = list(self._packets)
        self._packets.clear()
        self._update_status_events()
        self.events.has_changed._trigger()
        return packets

In [ ]:
#|hide
show_doc(fbdev.node.Node)

---

### Node

>      Node (node_spec:fbdev.graph.NodeSpec, parent_graph_process)

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class Node(AddressableMixin):
    _address_separator = '>'
    
    def __init__(self, node_spec:NodeSpec, parent_graph_process:GraphComponentFactory=None):
        if node_spec.component_type.is_factory:
            raise ValueError("Node cannot have a component type that is a ComponentFactory.")
        if parent_graph_process is not None and not issubclass(parent_graph_process.__class__, GraphComponentFactory):
            raise ValueError("Node must have a parent subclassed from GraphComponentFactory.")
        
        self._graph_elem_type = 'NODE'
        
        self._parent_graph_process:GraphComponentFactory = parent_graph_process
        self._node_spec:NodeSpec = node_spec
        self._component_process:BaseComponent = None
        self._task_manager:TaskManager = TaskManager(self)
        
        self._packet_handler:PacketHandler = None
        self._packet_registry:Union[PacketRegistry, None] = None # Is only not None if the node is the net.
        
        self._states:StateCollection = StateCollection()
        self._states._add_state(StateHandler("initialised", False), readonly=True)
        self._states._add_state(StateHandler("configured", False), readonly=True)
        self._states._add_state(StateHandler("running", False), readonly=True)
        self._states._add_state(StateHandler("stopping", False), readonly=True)
        self._states._add_state(StateHandler("component_executing", False), readonly=True)

        self._events:EventCollection = EventCollection()
        self._events._add_event(EventHandler("stopped_running"))

        self._edge_buses:Dict[Tuple[PortType, str], asyncio.Task] = {}
        self._edge_bus_resting:Dict[Tuple[PortType, str], asyncio.Event] = {}
        self._config_port_handlers:Dict[str, asyncio.Task] = {}
        
    @property
    def parent_graph_process(self): return self._parent_graph_process
    @property
    def parent(self): return self._parent_graph_process._parent_node if self._parent_graph_process is not None else None
    @property
    def is_net(self): return self.parent_graph_process is None
    @property
    def node_spec(self): return self._node_spec
    @property
    def component_type(self): return self.node_spec.component_type
    @property
    def component_process(self): return self._component_process
    @property
    def port_specs(self): return self.node_spec.component_type.port_specs
    @property
    def ports(self): return self.component_process.ports
    @property
    def states(self): return self._states
    @property
    def events(self): return self._events
    
    @property
    def packet_registry(self):
        if self.is_net: return self._packet_registry
        else: return self.parent.packet_registry
    @property
    def packet_handler(self): return self._packet_handler
    
    @property
    def id(self): return self.node_spec.id if not self.is_net else Graph.GRAPH_ID
    @property
    def rich_id(self): return self.node_spec.rich_id
    
    @property
    def incoming_edges(self):
        input_edges = {}
        if self.parent_graph_process is None: return {}
        for port_spec in self.port_specs.iter_ports():
            key = (self.id, port_spec.port_type, port_spec.name)
            if key in self.parent_graph_process.graph.head_connections:
                edge_id = self.parent_graph_process.graph.head_connections[key]
                input_edges[(port_spec.port_type, port_spec.name)] = self.parent_graph_process.edges[edge_id]
        return input_edges
    @property
    def outgoing_edges(self):
        output_edges = {}
        if self.parent_graph_process is None: return {}
        for port_spec in self.port_specs.iter_ports():
            key = (self.id, port_spec.port_type, port_spec.name)
            if key in self.parent_graph_process.graph.tail_connections:
                edge_id = self.parent_graph_process.graph.tail_connections[key]
                output_edges[(port_spec.port_type, port_spec.name)] = self.parent_graph_process.edges[edge_id]
        return output_edges
    @property
    def adjacent_edges(self):
        return {**self.incoming_edges, **self.outgoing_edges}
    
    def _handle_component_process_exception(self, task:asyncio.Task, exception:Exception, source_trace:Tuple[Union[BaseComponent], ...]):
        try:
            raise ComponentError() from exception
        except ComponentError as e:
            if not self.states.stopping.get(): self.stop()
            self._task_manager.submit_exception(task, e, source_trace)
    
    def initialise(self):
        if self.is_net: self._packet_registry = PacketRegistry()
        self._packet_handler = PacketHandler(registry=self.packet_registry, parent_node=self)
        self._component_process = self.component_type()
        self._component_process._parent_node = self
        self._component_process._task_manager.subscribe(self._handle_component_process_exception)
        self.states._initialised.set(True)
        self.states._configured.set(self.component_process.check_configured())
        
    def reset(self):
        self._component_process = None
        self.states._initialised.set(False)
        self.states._configured.set(False)
                
    async def _handle_in_edge_port_connection(self, edge:Edge, port:BasePort, is_resting:asyncio.Event):
        try:
            edge_non_empty = edge.states.empty.get_state_event(False)
            while True:
                await await_multiple_events(port._requesting_packet, edge_non_empty)
                is_resting.clear()
                packet = edge._unload()
                port._load_packet(packet)
                self._packet_handler._register_move(packet, self, port.port_type, port.name)
                port._received_packet.set()
                is_resting.set()
                await asyncio.sleep(0)
        except asyncio.CancelledError: pass
            
    async def _handle_out_edge_port_connection(self, edge:Edge, port:BasePort, is_resting:asyncio.Event):
        try:
            edge_non_full = edge.states.full.get_state_event(False)
            while True:
                await await_multiple_events(port._ready_to_unload, edge_non_full)
                is_resting.clear()
                packet = port._unload_packet()
                edge._load(packet)
                self._packet_handler._register_move(packet, edge, port.port_type, port.name)
                port._package_sent.set()
                is_resting.set()
                await asyncio.sleep(0)
        except asyncio.CancelledError: pass
        
    async def _handle_component_config(self, config_port):
        """Continually awaits receiving packets at a config port, and updates the component process's config."""
        try:
            while True:
                packet = await config_port._receive()
                await self.states.component_executing.wait(False) # Doing the awaits in this order guarantees that a config wont be updated whilst the component is executing (though it can be received during it).
                await self.component_process.set_config(config_port.name, packet)
                packet = None
                self.states._configured.set(self.component_process.check_configured())
                await asyncio.sleep(0)
        except asyncio.CancelledError: pass
        finally:
            if packet is not None:
                raise LostPacketError(packet, f"Packet lost in {self.__class__.__name__}._handle_component_config. Id: {self.rich_address}")
        
    def _start_edge_bus(self, port_type:PortType, port_name:str, edge:Edge):
        port = self.component_process.ports[port_type][port_name]
        is_resting = asyncio.Event()
        if port_type.is_input_port:
            edge_bus = self._task_manager.create_task(self._handle_in_edge_port_connection(edge, port, is_resting))
        else:
            edge_bus = self._task_manager.create_task(self._handle_out_edge_port_connection(edge, port, is_resting))
        self._edge_buses[(port_type, port_name)] = edge_bus
        self._edge_bus_resting[(port_type, port_name)] = is_resting
        
    async def _stop_edge_bus(self, port_type:PortType, port_name:str):
        await self._task_manager.cancel_wait(self._edge_buses[(port_type, port_name)])
        del self._edge_buses[(port_type, port_name)]
        del self._edge_bus_resting[(port_type, port_name)]
        
    def _start_config_handler(self, port_name:str):
        port = self.component_process.ports[PortType.CONFIG][port_name]
        config_task = self._task_manager.create_task(self._handle_component_config(port))
        self._config_port_handlers[port_name] = config_task
        
    async def _stop_config_handler(self, port_name:str):
        await self._task_manager.cancel_wait(self._config_port_handlers[port_name])
        
    async def send_input(self, port_type:PortType, port_name:str, data:Any):
        if not port_type.is_input_port: raise ValueError(f"Port type {port_type} is not an input port.")
        port = self.component_process.ports[port_type][port_name]
        if (port_type, port_name) in self._edge_buses:
            await await_multiple_events(port._requesting_packet, self._edge_bus_resting[(port_type, port_name)])
        else:
            await port._requesting_packet.wait()
        packet = self._packet_handler._create(data)
        port._load_packet(packet)
        port._received_packet.set()
        
    async def receive_output(self, port_type:PortType, port_name:str) -> Any:
        if port_type.is_input_port: raise ValueError(f"Port type {port_type} is not an output port.")
        port = self.component_process.ports[port_type][port_name]
        if (port_type, port_name) in self._edge_buses:
            await await_multiple_events(port._ready_to_unload, self._edge_bus_resting[(port_type, port_name)])
        else:
            await port._ready_to_unload.wait()
        packet = port._unload_packet()
        port._package_sent.set()
        data = await self._packet_handler._consume(packet)
        return data
                
    def run(self):
        self._run_task = self._task_manager.create_task(self._run())
        
    async def _run(self):
        if not self.states.initialised.get(): raise RuntimeError("Node is not initialised.")
        if self.states.running.get(): raise RuntimeError("Node is already running.")
        self.states._running.set(True)
        self.states._stopping.set(False)
    
        # Run component
        try:
            # Start edge buses
            for (port_type, port_name), edge in self.adjacent_edges.items():
                self._start_edge_bus(port_type, port_name, edge)
            
            # Start config handlers
            for port_name in self.port_specs[PortType.CONFIG]:
                self._start_config_handler(port_name)
                
            await self.component_process.start_background_tasks()
            while True:
                await self.states.configured.wait(True)
                if self.component_process.is_passive:
                    await asyncio.get_running_loop().create_future() # Wait forever
                else:
                    self.states._component_executing.set(True)
                    await self.component_process.execute()
                    self.states._component_executing.set(False)
                    await asyncio.sleep(0)
        except asyncio.CancelledError: # TODO should this be here? I'm not sure if it is a good idea to catch CancelledError without doing anything.
            pass
        finally:
            await self.component_process.stop_background_tasks()
            # Stop edge buses
            for port_type, port_name in list(self._edge_buses.keys()):
                await self._stop_edge_bus(port_type, port_name)
            
            # Stop config handlers
            for port_name in list(self.port_specs[PortType.CONFIG].keys()):
                await self._stop_config_handler[port_name]
                
            self.states._running.set(False)
            self.states._stopping.set(False)
             
    async def _execute_with_exception_monitoring(self, *coros: List[Coroutine], print_all_exceptions=True):
        """Run a coroutine and monitor for exceptions in the coroutine, as well as
        any exceptions that occurs in the task manager. Therefore, for it to work
        as expected, the coroutine must be starting tasks using self._task_manager.create_task().
        """
        async def all_coros():
            await asyncio.gather(*[asyncio.create_task(coro) for coro in coros])
        task = asyncio.create_task(all_coros())
        monitor_task = asyncio.create_task(self._task_manager.wait_for_exceptions())
        await asyncio.wait([task, monitor_task], return_when=asyncio.FIRST_COMPLETED)
        exceptions = self._task_manager.get_exceptions()
        if task.done():
            try: await task
            except Exception as e: exceptions.append((task, e, ()))
        if not monitor_task.done():
            monitor_task.cancel()
        
        if print_all_exceptions:
            for i, (task, e, source_trace) in enumerate(exceptions):
                msg = f"Message: {e}\n\n{''.join(traceback.format_exception(type(e), e, e.__traceback__))}\n\n"
                msg = "\n".join([f"    {line}" for line in msg.split("\n")])
                print(f"Exception {i+1} ({e.__class__.__name__}):")
                print(msg)
                
        for task, e, source_trace in exceptions:
            raise e
             
    def stop(self):
        if not self.states.running.get(): raise RuntimeError("Node is not running.")
        if self.states.stopping.get(): raise RuntimeError("Node is already stopping.")
        self.states._stopping.set(True)
        self._task_manager.cancel(self._run_task)
             
    async def async_stop(self):
        self.stop()
        try:
            await self.states.running.wait(False)
        except asyncio.CancelledError:
            print(asyncio.current_task())
            raise
        
    async def destroy(self):
        if self.states.running.get(): await self.async_stop()
        await self.component_process.destroy()
        await self._task_manager.destroy()
        self.reset()

In [ ]:
#|hide
show_doc(fbdev.node.GraphComponentFactory)

---

### GraphComponentFactory

>      GraphComponentFactory ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class GraphComponentFactory(ComponentFactory):
    _graph:Graph = None # Set by `get_component`
    graph:ReadonlyGraph = None # Set by `get_component`
    
    is_passive = True
    contains_graph = True
    
    def __init__(self):
        super().__init__()
        
        self._edges: Dict[Hashable, Edge] = {}
        self._nodes: Dict[Hashable, Node] = {}
        
        self._in_buses: List[asyncio.Task] = []
        self._out_buses: List[asyncio.Task] = []
        self._config_buses: List[asyncio.Task] = []
        
        # Add edges
        for edge_spec in self.graph.edges.values():
            self._edges[edge_spec.id] = Edge(edge_spec, self)
        # Add nodes
        for node_spec in self.graph.nodes.values():
            node =  Node(node_spec, self)
            self._nodes[node_spec.id] = node
            node._task_manager.subscribe(self._handle_child_node_exception)

    @property
    def edges(self): return MappingProxyType(self._edges)
    @property
    def nodes(self): return MappingProxyType(self._nodes)
    
    def _handle_child_node_exception(self, task:asyncio.Task, exception:Exception, source_trace:Tuple[Node, ...]):
        try:
            raise NodeError() from exception
        except NodeError as e:
            self._task_manager.submit_exception(task, e, source_trace)
    
    @classmethod
    def get_component(cls, graph, component_name=None):
        port_specs = graph.port_specs
        return cls._create_component_class(component_name=component_name, class_attrs={
            'port_specs' : port_specs,
            '_graph' : graph,
            'graph' : ReadonlyGraph(graph)
        })
    
    async def _handle_graph_in_port(self, graph_port:InputPort, edge:Edge):
        edge_non_full = edge.states.full.get_state_event(False)
        packet = None
        try:
            while True:
                packet = await graph_port.receive()
                await edge_non_full.wait()
                edge._load(packet)
                self._parent_node._packet_handler._register_move(packet, edge, graph_port.port_type, graph_port.name)
                packet = None
                await asyncio.sleep(0)
        except asyncio.CancelledError: pass
        finally:
            if packet is not None:
                raise LostPacketError(packet, f"Packet lost in {self.__class__.__name__}._handle_graph_in_port. Id: {self._parent_node.rich_address}")
            
    async def _handle_graph_out_port(self, graph_port:OutputPort, edge:Edge):
        edge_non_empty = edge.states.empty.get_state_event(False)
        packet = None
        try:
            while True:
                await edge_non_empty.wait()
                packet = edge._unload()
                self._parent_node._packet_handler._register_move(packet, self._parent_node, graph_port.port_type, graph_port.name)
                # In-between the unload from the edge, and the outputting into the output port, the location of the packet is in the subnet.
                await graph_port.put(packet)
                packet = None
                await asyncio.sleep(0)
        except asyncio.CancelledError: pass
        finally:
            if packet is not None:
                raise LostPacketError(packet, f"Packet lost in {self.__class__.__name__}._handle_graph_out_port. Id: {self._parent_node.rich_address}")
        
    async def _forward_config_into_graph(self, edge:Edge, packet:Packet):
        try:
            await edge.states.full.get_state_event(False)
        except asyncio.CancelledError: pass
        finally:
            if not edge.states.full.get():
                edge._load(packet) #TODO TBC on whether it is a good idea to do a last ditch effort to load the packet
            else:
                raise LostPacketError(packet, f"Packet lost in {self.__class__.__name__}._forward_config_into_graph. Id: {self._parent_node.rich_address}")
        
    def set_config(self, name:str, packet:Packet):
        """Overloads BaseComponent.set_config to forward the config to child nodes, where such edges have been defined."""
        if (PortType.CONFIG, name) in self.graph.graph_tail_connections:
            edge_id = self.graph.graph_tail_connections[(PortType.CONFIG, name)]
            task = self._task_manager.create_task(self._forward_config_into_graph(self.edges[edge_id], packet))
            self._config_buses.append(task)
        else:
            super().set_config(name, packet)
    
    async def execute(self):
        await super().execute()
    
    async def start_background_tasks(self):
        # Don't catch CancelledError here, as it should never be cancelled, and if it is the exception should be raised.
        for (port_type, port_name), edge_id in self.graph.graph_tail_connections.items():
            if port_type == PortType.CONFIG:
                continue 
            task = self._task_manager.create_task(self._handle_graph_in_port(self.ports[port_type][port_name], self.edges[edge_id]))
            self._in_buses.append(task)
            
        for (port_type, port_name), edge_id in self.graph.graph_head_connections.items():
            task = self._task_manager.create_task(self._handle_graph_out_port(self.ports[port_type][port_name], self.edges[edge_id]))
            self._out_buses.append(task)
        
        for node in self.nodes.values():
            if not node.states.initialised.get():
                node.initialise() # TODO allow for more customisable node initialisation
            node.run()
            
    async def stop_background_tasks(self):
        # Don't catch CancelledError here, as it should never be cancelled, and if it is the exception should be raised.
        for task in self._in_buses+self._out_buses+self._config_buses:
            await self._task_manager.cancel_wait(task)
        self._in_buses.clear()
        self._out_buses.clear()
        self._config_buses.clear()
        for node in self.nodes.values():
            if node.states.running.get():
                await node.async_stop()
    
    async def destroy(self):
        await super().destroy()
        for node in self.nodes.values():
            await node.destroy()
